In [1]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=20)

def parses(input):
    def parse_line(line):
        mod, dsts = line.split(' -> ')
        dsts = dsts.split(', ')
        if mod[0] in '%&':
            kind = {'%': 'flip', '&': 'conj'}[mod[0]]
            mod = mod[1:]
        elif mod == 'broadcaster':
            kind = 'cast'
        return mod, (kind, dsts)
    return [parse_line(line) for line in input.strip().split('\n')]

data = parses(puzzle.input_data)

In [2]:
sample = parses("""broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
""")

sample2 = parses("""broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output""")

In [8]:
from collections import deque
import math
import itertools

def solve_a(data):
    mods = dict(data)

    flip_states = {mod: 0 for mod, (kind, _) in data if kind == 'flip'}
    conj_memory = {mod: {} for mod, (kind, _) in data if kind == 'conj'}
    for mod, (kind, connected) in data:
        for dst in connected:
            if dst in conj_memory:
                conj_memory[dst][mod] = 0

    pulses = [0,0]

    for i in range(1000):
        queue = deque(
            [('broadcaster', dst, 0) for dst in mods['broadcaster'][1]]
        )
        pulses[0] += 1+len(queue) # button + broadcasts

        while queue:
            src, mod, pulse = queue.popleft()
            if mod not in mods:
                continue
                
            kind, connected = mods[mod]
            if kind == 'flip':
                if pulse == 1: 
                    continue
                flip_states[mod] ^= 1
                new_pulse = flip_states[mod]
            elif kind == 'conj':
                mem = conj_memory[mod]
                mem[src] = pulse
                new_pulse = 1 ^ all(s==1 for s in mem.values())

            for dst in connected:
                pulses[new_pulse] += 1
                queue.append((mod, dst, new_pulse))
                
    return math.prod(pulses)

In [9]:
solve_a(sample) == 32000000

True

In [10]:
solve_a(sample2) == 11687500

True

In [31]:
from collections import deque
import math
import itertools


### The puzzle input is structured with 4 separate binary counters that 
# count up to prime numbers and then reset to zero, thus we just 
# need to find the periods of the high pulses going into the conjunction
# gate before rx
def solve_b(data):
    mods = dict(data)

    flip_states = {mod: 0 for mod, (kind, _) in data if kind == 'flip'}
    conj_memory = {mod: {} for mod, (kind, _) in data if kind == 'conj'}
    for mod, (kind, connected) in data:
        for dst in connected:
            if dst in conj_memory:
                conj_memory[dst][mod] = 0
    
    # find the nodes 
    mods = dict(data)
    reverse = {}
    for mod, (kind, connected) in data:
        for dst in connected:
            reverse[dst] = reverse.get(dst, []) + [mod]
    prev = reverse['rx'][0]
    required = reverse[prev]
    
    periods = {}

    for i in itertools.count(1):
        queue = deque(
            [('broadcaster', dst, 0) for dst in mods['broadcaster'][1]]
        )

        while queue:
            src, mod, pulse = queue.popleft()
            
            if mod == prev and pulse == 1:
                print(periods)
                periods[src] = i
                if len(periods) == 4:
                    return math.lcm(*periods.values())
            
            if mod not in mods:
                continue

            kind, connected = mods[mod]
            if kind == 'flip':
                if pulse == 1: 
                    continue
                flip_states[mod] ^= 1
                new_pulse = flip_states[mod]
            elif kind == 'conj':
                mem = conj_memory[mod]
                mem[src] = pulse
                new_pulse = 1 ^ all(s==1 for s in mem.values())

            for dst in connected:
                queue.append((mod, dst, new_pulse))
                
    

In [33]:
solve_b(data) == 227411378431763

{}
{'lt': 3739}
{'lt': 3739, 'qh': 3821}
{'lt': 3739, 'qh': 3821, 'bq': 3889}


True